In [7]:
# import modules
from pypet import Environment, Parameter, cartesian_product, progressbar, Trajectory
import numpy as np
import csv
import os
import copy
import pickle
import logging
from datetime import date

In [5]:
# load data from one simulation:

repository_path = os.path.normpath('C:/Users/julia/Documents/Resources/InfluenzaFitnessLandscape/NewApproachFromMarch2021/'
                                   'InfluenzaFitnessInference')
result_directory = repository_path
simu_date = '2021Apr07'


simu_file = os.path.join(result_directory, 'results', 'simulations', simu_date + '.hdf5')
# only need trajectory, not environment to look at parameters and results:
traj = Trajectory(simu_date, add_time=False)
# load the trajectory from the file with only parameters but not results loaded
traj.f_load(filename=simu_file, load_parameters=2,
            load_results=0, load_derived_parameters=0)

temp_folder = os.path.join(result_directory, 'results', 'simulations', simu_date + '_temp')
test_filename = 'running_N_pop_1e+06N_site_20N_state_2mu_1e-04sigma_h_1D0_5h_0_-7J_0_0seed_123456N_simu_2e+02.data'
test_filepath = os.path.join(temp_folder, test_filename)

with open(test_filepath, 'rb') as f:
    resulting_data = pickle.load(f)
    
strain_yearly = resulting_data['strain_yearly']
strain_frequency_yearly = resulting_data['strain_frequency_yearly']